#### blip 사용해서 캡션 만들기

In [ ]:
import os
import json
from PIL import Image
import torch
from tqdm import tqdm
from transformers import AutoProcessor, Blip2ForImageTextRetrieval

# ====== 경로 설정 ======
def setup_paths():
    # 이미지 폴더 경로 설정
    image_base_dir = "/content/drive/MyDrive/컨퍼런스/스타일추천/한국인얼굴/celeb"
    output_dir = "/content/drive/MyDrive/컨퍼런스/스타일추천/한국인얼굴/korean_celebrity_makeup_output"

    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(image_base_dir):
        raise FileNotFoundError(f"이미지 폴더를 찾을 수 없습니다: {image_base_dir}")

    return image_base_dir, output_dir

# ====== 키워드 카테고리 정의 ======
categories = {
    "분위기": [
        "꾸안꾸", "내추럴", "데일리", "세련됨", "모던", "청순", "러블리", "차분",
        "시크", "트렌디", "우아", "럭셔리", "발랄", "청량", "페미닌", "섹시",
        "심플", "밝은", "은은한", "깔끔", "고급스러운", "쿨", "매혹적",
        "귀여운", "깜찍", "사랑스러운", "활기찬", "상큼", "성숙", "클래식"
    ],
    "눈": [
        "내추럴아이", "스모키아이", "세미스모키", "글리터아이", "매트아이",
        "브라운아이", "핑크아이", "코랄아이", "오렌지아이",
        "언더라인", "또렷한라인", "자연스러운라인", "음영"
    ],
    "피부": [
        "광채피부", "물광피부", "윤기피부", "매트피부", "보송피부", "수분피부",
        "촉촉피부", "투명피부", "글로우", "하이라이트", "컨투어", "쉐딩", "톤업"
    ],
    "블러셔": [
        "핑크블러셔", "코랄블러셔", "피치블러셔", "오렌지블러셔"
    ],
    "입술": [
        "레드립", "핑크립", "코랄립", "누드립", "브라운립", "오렌지립", "로즈립",
        "매트립", "글로시립", "그라데이션립", "자연스러운립", "또렷한립라인"
    ],
    "눈썹": [
        "자연스러운눈썹", "아치눈썹", "일자눈썹", "진한눈썹", "연한눈썹"
    ],
    "속눈썹": [
        "자연스러운속눈썹", "볼륨속눈썹", "긴속눈썹", "컬링속눈썹", "볼륨마스카라", "롱래쉬마스카라"
    ],
    "톤": [
        "웜톤", "쿨톤", "뉴트럴톤", "봄웜", "여름쿨", "가을웜", "겨울쿨"
    ]
}

# 한영 번역 딕셔너리
keyword_translation = {
    "꾸안꾸": "effortless style", "내추럴": "natural beauty", "데일리": "everyday look",
    "세련됨": "sophisticated style", "모던": "modern style", "청순": "innocent beauty",
    "러블리": "lovely charm", "차분": "calm elegance", "시크": "chic style",
    "트렌디": "trendy look", "우아": "elegant style", "럭셔리": "luxury look",
    "발랄": "bright energy", "청량": "fresh beauty", "페미닌": "feminine beauty",
    "섹시": "sexy appeal", "심플": "simple style", "밝은": "bright mood",
    "은은한": "subtle beauty", "깔끔": "clean look", "고급스러운": "luxurious style",
    "쿨": "cool style", "매혹적": "alluring charm", "귀여운": "cute style",
    "깜찍": "adorable look", "사랑스러운": "lovable beauty", "활기찬": "energetic look",
    "상큼": "fresh charm", "성숙": "mature elegance", "클래식": "classic style",
    "내추럴아이": "natural eyes", "스모키아이": "smoky eyes", "세미스모키": "semi-smoky eyes",
    "글리터아이": "glitter eyes", "매트아이": "matte eyes", "브라운아이": "brown eyeshadow",
    "핑크아이": "pink eyeshadow", "코랄아이": "coral eyeshadow", "오렌지아이": "orange eyeshadow",
    "언더라인": "lower eyeliner", "또렷한라인": "defined eyeliner", "자연스러운라인": "natural eyeliner",
    "음영": "eye shadow depth", "광채피부": "glowing skin", "물광피부": "dewy glass skin",
    "윤기피부": "radiant skin", "매트피부": "matte skin", "보송피부": "soft velvet skin",
    "수분피부": "hydrated skin", "촉촉피부": "moist skin", "투명피부": "clear skin",
    "글로우": "glow effect", "하이라이트": "highlight", "컨투어": "contour", "쉐딩": "shading",
    "톤업": "tone-up effect", "핑크블러셔": "pink blush", "코랄블러셔": "coral blush",
    "피치블러셔": "peach blush", "오렌지블러셔": "orange blush", "레드립": "red lips",
    "핑크립": "pink lips", "코랄립": "coral lips", "누드립": "nude lips",
    "브라운립": "brown lips", "오렌지립": "orange lips", "로즈립": "rose lips",
    "매트립": "matte lips", "글로시립": "glossy lips", "그라데이션립": "gradient lips",
    "자연스러운립": "natural lips", "또렷한립라인": "defined lip line",
    "자연스러운눈썹": "natural brows", "아치눈썹": "arched brows", "일자눈썹": "straight brows",
    "진한눈썹": "bold brows", "연한눈썹": "light brows", "자연스러운속눈썹": "natural lashes",
    "볼륨속눈썹": "volume lashes", "긴속눈썹": "long lashes", "컬링속눈썹": "curled lashes",
    "볼륨마스카라": "volume mascara", "롱래쉬마스카라": "lengthening mascara",
    "웜톤": "warm tone", "쿨톤": "cool tone", "뉴트럴톤": "neutral tone",
    "봄웜": "spring warm", "여름쿨": "summer cool", "가을웜": "autumn warm", "겨울쿨": "winter cool"
}

# ====== 문장 생성 함수 ======
def make_english_sentence(keywords):
    """영어 키워드들을 자연스러운 문장으로 변환"""
    if not keywords:
        return ""

    if len(keywords) == 1:
        return f"This makeup look features {keywords[0]}."
    elif len(keywords) == 2:
        return f"This makeup look features {keywords[0]} and {keywords[1]}."
    else:
        return f"This makeup look features {', '.join(keywords[:-1])}, and {keywords[-1]}."

def make_korean_sentence(keywords):
    """한글 키워드들을 자연스러운 문장으로 변환"""
    if not keywords:
        return ""

    if len(keywords) == 1:
        return f"이 메이크업은 {keywords[0]} 스타일입니다."
    elif len(keywords) == 2:
        return f"이 메이크업은 {keywords[0]}과 {keywords[1]}이 특징입니다."
    else:
        return f"이 메이크업은 {', '.join(keywords[:-1])}, 그리고 {keywords[-1]}이 특징입니다."

# ====== BLIP2 모델 초기화 ======
def initialize_blip2_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = AutoProcessor.from_pretrained("Salesforce/blip2-itm-vit-g")
    model = Blip2ForImageTextRetrieval.from_pretrained(
        "Salesforce/blip2-itm-vit-g",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)
    return processor, model, device

# ====== 이미지 경로 수집 ======
def collect_image_paths(base_dir):
    image_paths = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')) and not file.startswith('._'):
                image_paths.append(os.path.join(root, file))
    return image_paths

# ====== 유사도 계산 ======
def calculate_similarities(image, processor, model, device):
    results = {}

    for cat_name, korean_keywords in categories.items():
        english_texts = [keyword_translation[kw] for kw in korean_keywords if kw in keyword_translation]

        if not english_texts:
            continue

        try:
            inputs = processor(images=image, text=english_texts, return_tensors="pt", padding=True, truncation=True).to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                probs = outputs.logits_per_image.softmax(dim=1)[0]

            best_idx = torch.argmax(probs).item()
            results[cat_name] = {
                "korean_keyword": korean_keywords[best_idx],
                "english_keyword": english_texts[best_idx],
                "score": probs[best_idx].item()
            }
        except:
            continue

    return results

# ====== 캡션 생성 함수들 ======
def generate_mood_caption(category_results):
    """분위기 + 디테일 2개 캡션 생성"""
    detail_categories = ["눈", "피부", "블러셔", "입술", "눈썹", "속눈썹"]
    details = [category_results[cat] for cat in detail_categories if cat in category_results]
    details.sort(key=lambda x: x["score"], reverse=True)

    if "분위기" not in category_results or len(details) < 2:
        return None

    mood = category_results["분위기"]
    korean_keywords = [mood["korean_keyword"], details[0]["korean_keyword"], details[1]["korean_keyword"]]
    english_keywords = [mood["english_keyword"], details[0]["english_keyword"], details[1]["english_keyword"]]

    return {
        "korean": f"{mood['korean_keyword']}, {details[0]['korean_keyword']}, {details[1]['korean_keyword']}",
        "english": f"{mood['english_keyword']}, {details[0]['english_keyword']}, {details[1]['english_keyword']}",
        "sentence_korean": make_korean_sentence(korean_keywords),
        "sentence_english": make_english_sentence(english_keywords),
        "components": {
            "mood": mood,
            "detail1": details[0],
            "detail2": details[1]
        }
    }

def generate_tone_caption(category_results):
    """톤 + 디테일 2개 캡션 생성"""
    detail_categories = ["눈", "피부", "블러셔", "입술", "눈썹", "속눈썹"]
    details = [category_results[cat] for cat in detail_categories if cat in category_results]
    details.sort(key=lambda x: x["score"], reverse=True)

    if "톤" not in category_results or len(details) < 2:
        return None

    tone = category_results["톤"]
    korean_keywords = [tone["korean_keyword"], details[0]["korean_keyword"], details[1]["korean_keyword"]]
    english_keywords = [tone["english_keyword"], details[0]["english_keyword"], details[1]["english_keyword"]]

    return {
        "korean": f"{tone['korean_keyword']}, {details[0]['korean_keyword']}, {details[1]['korean_keyword']}",
        "english": f"{tone['english_keyword']}, {details[0]['english_keyword']}, {details[1]['english_keyword']}",
        "sentence_korean": make_korean_sentence(korean_keywords),
        "sentence_english": make_english_sentence(english_keywords),
        "components": {
            "tone": tone,
            "detail1": details[0],
            "detail2": details[1]
        }
    }

# ====== 최종 포맷 변환 ======
def convert_to_final_format(result, caption_data):
    """API 응답 형태로 변환"""
    request = {
        "이미지경로": result["image_path"]
    }

    response = {
        "predicted_keywords": [],
        "caption": "",
        "prompt_en": ""
    }

    if caption_data:
        components = caption_data.get("components", {})

        # predicted_keywords 생성
        for comp_key, comp_value in components.items():
            if isinstance(comp_value, dict):
                keyword_entry = {
                    "keyword_kr": comp_value.get("korean_keyword", ""),
                    "keyword_en": comp_value.get("english_keyword", ""),
                    "score": round(comp_value.get("score", 0.0), 4)
                }
                response["predicted_keywords"].append(keyword_entry)

        # caption과 prompt_en 생성
        keywords_en = [kw["keyword_en"] for kw in response["predicted_keywords"]]
        if len(keywords_en) == 1:
            response["caption"] = f"A woman with {keywords_en[0]} makeup."
        elif len(keywords_en) == 2:
            response["caption"] = f"A woman with {keywords_en[0]} and {keywords_en[1]} makeup."
        elif len(keywords_en) >= 3:
            response["caption"] = f"A woman with {', '.join(keywords_en[:-1])}, and {keywords_en[-1]} makeup."

        if keywords_en:
            response["prompt_en"] = f"{', '.join(keywords_en)} makeup, suitable for natural beauty."

    return {
        "request": request,
        "response": response
    }

# ====== 메인 실행 ======
def main():
    print("메이크업 캡션 생성기 시작!")

    # 경로 설정
    print("경로 설정 중...")
    image_base_dir, output_dir = setup_paths()
    print(f"이미지 폴더: {image_base_dir}")
    print(f"출력 폴더: {output_dir}")

    # 모델 초기화
    print("\nBLIP2 모델 로딩 중...")
    processor, model, device = initialize_blip2_model()
    print(f"모델 로딩 완료! 디바이스: {device}")

    # 이미지 수집
    print("\n이미지 파일 수집 중...")
    image_paths = collect_image_paths(image_base_dir)
    total_images = len(image_paths)
    print(f"총 {total_images}개 이미지 발견")

    if total_images == 0:
        print("처리할 이미지가 없습니다.")
        return

    # 연예인별 이미지 개수 확인
    celeb_count = {}
    for path in image_paths:
        celeb_name = os.path.basename(os.path.dirname(path))
        celeb_count[celeb_name] = celeb_count.get(celeb_name, 0) + 1

    print(f"연예인 수: {len(celeb_count)}명")
    for celeb, count in list(celeb_count.items())[:5]:  # 상위 5명만 출력
        print(f"  - {celeb}: {count}개")
    if len(celeb_count) > 5:
        print(f"  ... 외 {len(celeb_count)-5}명")

    # 이미지 처리
    print(f"\n이미지 처리 시작 ({total_images}개)")
    mood_results = []
    tone_results = []
    mood_final_format = []
    tone_final_format = []

    for i, image_path in enumerate(tqdm(image_paths, desc="Processing images")):
        try:
            # 진행률 출력 (10개마다)
            if (i + 1) % 10 == 0:
                progress = ((i + 1) / total_images) * 100
                print(f"\n진행률: {i+1}/{total_images} ({progress:.1f}%)")
                print(f"현재까지 - 분위기 캡션: {len(mood_results)}개, 톤 캡션: {len(tone_results)}개")

            image = Image.open(image_path).convert("RGB")
            category_results = calculate_similarities(image, processor, model, device)

            relative_path = os.path.relpath(image_path, os.path.dirname(output_dir))

            base_result = {
                "image_path": relative_path,
                "image_name": os.path.basename(image_path),
                "celebrity_name": os.path.basename(os.path.dirname(image_path)),
                "category_results": category_results
            }

            # 분위기 + 디테일 2개 캡션
            mood_caption = generate_mood_caption(category_results)
            if mood_caption:
                mood_result = base_result.copy()
                mood_result["caption"] = mood_caption
                mood_results.append(mood_result)

                # 최종 포맷 변환
                mood_final = convert_to_final_format(mood_result, mood_caption)
                mood_final_format.append(mood_final)

            # 톤 + 디테일 2개 캡션
            tone_caption = generate_tone_caption(category_results)
            if tone_caption:
                tone_result = base_result.copy()
                tone_result["caption"] = tone_caption
                tone_results.append(tone_result)

                # 최종 포맷 변환
                tone_final = convert_to_final_format(tone_result, tone_caption)
                tone_final_format.append(tone_final)

            # GPU 메모리 정리 (50개마다)
            if (i + 1) % 50 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"\nGPU 메모리 정리 완료")

        except Exception as e:
            print(f"\n에러 발생 ({os.path.basename(image_path)}): {e}")
            continue

    # 결과 저장
    print("\n결과 저장 중...")

    # 1. 분위기 기반 결과들
    mood_detailed_path = os.path.join(output_dir, "makeup_captions_mood_detailed.json")
    with open(mood_detailed_path, "w", encoding="utf-8") as f:
        json.dump(mood_results, f, indent=2, ensure_ascii=False)

    mood_final_path = os.path.join(output_dir, "makeup_captions_mood_final.json")
    with open(mood_final_path, "w", encoding="utf-8") as f:
        json.dump(mood_final_format, f, indent=2, ensure_ascii=False)

    # 2. 톤 기반 결과들
    tone_detailed_path = os.path.join(output_dir, "makeup_captions_tone_detailed.json")
    with open(tone_detailed_path, "w", encoding="utf-8") as f:
        json.dump(tone_results, f, indent=2, ensure_ascii=False)

    tone_final_path = os.path.join(output_dir, "makeup_captions_tone_final.json")
    with open(tone_final_path, "w", encoding="utf-8") as f:
        json.dump(tone_final_format, f, indent=2, ensure_ascii=False)

    print(f"완료!")
    print(f"분위기 기반 상세 결과: {mood_detailed_path} ({len(mood_results)}개)")
    print(f"분위기 기반 최종 결과: {mood_final_path} ({len(mood_final_format)}개)")
    print(f"톤 기반 상세 결과: {tone_detailed_path} ({len(tone_results)}개)")
    print(f"톤 기반 최종 결과: {tone_final_path} ({len(tone_final_format)}개)")

    # 샘플 출력
    if mood_results:
        print(f"\n분위기 기반 샘플:")
        sample = mood_results[0]
        print(f"이미지: {sample['image_name']}")
        print(f"키워드: {sample['caption']['korean']}")
        print(f"문장: {sample['caption']['sentence_korean']}")

    if tone_results:
        print(f"\n톤 기반 샘플:")
        sample = tone_results[0]
        print(f"이미지: {sample['image_name']}")
        print(f"키워드: {sample['caption']['korean']}")
        print(f"문장: {sample['caption']['sentence_korean']}")

if __name__ == "__main__":
    main()